In [1]:
import itertools
import pandas as pd

# Define parameter value sets
enterprise = ["Big", "Small"]
budget = ["High", "Medium", "Low"]
infra = ["New", "Old"]
legacy = ["Present", "Absent"]
deployment = ["Same server", "Different servers"]
direction = ["User→Server", "Server→User"]
bank_relation = ["Same bank", "Different bank"]

# Banks with attributes
banks = {
    "ICICI": {"Adoption": "PQC", "Algo": "PQC only", "Hardware": "New", "Size": "Big"},
    "HDFC": {"Adoption": "Hybrid", "Algo": "Classical + PQC", "Hardware": "Mid", "Size": "Big"},
    "SBI": {"Adoption": "Classical", "Algo": "Classical only", "Hardware": "Old", "Size": "Big"},
    "Saraswat": {"Adoption": "Classical", "Algo": "Classical only", "Hardware": "Latest", "Size": "Small"},
}

# Build all permutations (192 * 4 = 768)
cases = []
for bank, attrs in banks.items():
    for combo in itertools.product(enterprise, budget, infra, legacy, deployment, direction, bank_relation):
        cases.append((bank, attrs["Adoption"], attrs["Algo"], attrs["Hardware"], attrs["Size"], *combo))

# Put into DataFrame
columns = ["Bank", "Adoption", "Algo_pref", "Bank_HW", "Bank_Size",
           "Enterprise", "Budget", "Infra", "Legacy", "Deployment", "Direction", "Bank relation"]

test_cases_df = pd.DataFrame(cases, columns=columns)

print("Total test cases:", len(test_cases_df))  # should print 768
print(test_cases_df.head(12))

# Save to CSV
test_cases_df.to_csv("pqc_bank_test_cases.csv", index=False)
print("All 768 test cases saved to pqc_bank_test_cases.csv")


Total test cases: 768
     Bank Adoption Algo_pref Bank_HW Bank_Size Enterprise Budget Infra  \
0   ICICI      PQC  PQC only     New       Big        Big   High   New   
1   ICICI      PQC  PQC only     New       Big        Big   High   New   
2   ICICI      PQC  PQC only     New       Big        Big   High   New   
3   ICICI      PQC  PQC only     New       Big        Big   High   New   
4   ICICI      PQC  PQC only     New       Big        Big   High   New   
5   ICICI      PQC  PQC only     New       Big        Big   High   New   
6   ICICI      PQC  PQC only     New       Big        Big   High   New   
7   ICICI      PQC  PQC only     New       Big        Big   High   New   
8   ICICI      PQC  PQC only     New       Big        Big   High   New   
9   ICICI      PQC  PQC only     New       Big        Big   High   New   
10  ICICI      PQC  PQC only     New       Big        Big   High   New   
11  ICICI      PQC  PQC only     New       Big        Big   High   New   

     Legacy    

In [2]:
# PQC Test-case → Bank mapping (rule-based)
# Generates all permutations (192 cases) then maps to 4 banks and provides compatibility & recommendations.
# Output: DataFrame (192*4 rows) saved as 'pqc_testcases_bank_mapping.csv'

import itertools
import pandas as pd
import numpy as np

# -------------------------
# 1) Define test-case parameter spaces (same as before)
# -------------------------
enterprise = ["Big", "Small"]
budget = ["High", "Medium", "Low"]
infra = ["New", "Old"]
legacy = ["Present", "Absent"]
deployment = ["Same server", "Different servers"]
direction = ["User->Server", "Server->User"]
bank_relation = ["Same bank", "Different bank"]

params = [enterprise, budget, infra, legacy, deployment, direction, bank_relation]
columns = ["Enterprise", "Budget", "Infra", "Legacy", "Deployment", "Direction", "Bank relation"]

cases = list(itertools.product(*params))
test_cases_df = pd.DataFrame(cases, columns=columns)
print("Generated test case count:", len(test_cases_df))  # should be 192

# -------------------------
# 2) Define bank profiles
# -------------------------
# Profiles derived from your description:
# icici - adopt to pqc / new hardware / big bank
# hdfc  - hybrid (classical/pqc) / mid hardware / big bank
# sbi   - classical only / old hardware / low budget
# saraswat - classical / latest hardware / mid

bank_profiles = {
    "ICICI": {
        "policy": "adopt_pqc",
        "hw_pref": "new",
        "size": "Big",
        "budget_pref": "High",
        "hw_note": "supports high-end FPGA/ASIC; crypto-agile ready"
    },
    "HDFC": {
        "policy": "hybrid",
        "hw_pref": "mid",
        "size": "Big",
        "budget_pref": "Medium",
        "hw_note": "mid-tier FPGA/accelerator; supports hybrid stacks"
    },
    "SBI": {
        "policy": "classical_only",
        "hw_pref": "old",
        "size": "Low",
        "budget_pref": "Low",
        "hw_note": "legacy systems, minimal hardware upgrades"
    },
    "Saraswat": {
        "policy": "classical_but_latest_hw",
        "hw_pref": "latest",
        "size": "Mid",
        "budget_pref": "Medium",
        "hw_note": "policy classical, but hardware is modern (can run PQC)"
    }
}

# -------------------------
# 3) Algorithm & hardware recommendation pools
# -------------------------
# Simple recommended algorithm lists and typical characteristics
# You can expand these lists or link to concrete parameter sets later.

PQC_ALGOS = {
    "Kyber": {"type":"KEM", "key_size":"small-medium", "latency":"low-medium"},
    "Dilithium": {"type":"Signature", "key_size":"medium", "latency":"low"},
    "Falcon": {"type":"Signature", "key_size":"small", "latency":"very_low", "cost":"heavy_fpga"},
    "SPHINCS+": {"type":"Signature", "key_size":"large", "latency":"high"},
}

CLASSICAL_ALGOS = {
    "RSA": {"type":"KEM/Signature", "key_size":"large", "latency":"variable"},
    "ECDSA": {"type":"Signature", "key_size":"small", "latency":"low"}
}

HYBRID_RECOMMENDATION = ["Kyber+RSA-hybrid", "Dilithium+ECDSA-hybrid"]

# Hardware module descriptive options (not raw FPGA bitstreams — descriptive)
HW_MODULE_TEMPLATES = {
    "high_end_fpga": {"desc":"High-end FPGA / ASIC (lots of LUT/BRAM/DSP, high freq)", "suitable_for":"Falcon/Dilithium/Kyber"},
    "mid_tier_fpga": {"desc":"Mid-tier FPGA or crypto-accelerator (balanced resources)", "suitable_for":"Kyber/Dilithium"},
    "cpu_with_accel": {"desc":"Server CPU with crypto acceleration (AVX2/AVX512) or co-processor", "suitable_for":"Kyber/ECDSA"},
    "lightweight_cpu": {"desc":"Lightweight CPU (low-power) - prefer small/optimized algorithms", "suitable_for":"ECDSA / RSA (legacy)"},
    "latest_hw_but_policy_classical": {"desc":"Latest hardware available but policy prefers classical - PQC possible if policy changes", "suitable_for":"Optional PQC testing"}
}

# -------------------------
# 4) Scoring function (how well a bank fits a test-case)
# -------------------------
# Weighted rule-based scoring; result 0-100
def compatibility_score(testcase, bank_profile):
    # Weights
    w_enterprise = 0.15
    w_budget = 0.20
    w_infra = 0.25
    w_legacy = 0.10
    w_deployment = 0.05
    w_direction = 0.05
    w_bank_relation = 0.05
    w_policy = 0.15  # whether bank policy and test-case constraints align
    
    score = 0.0
    # Enterprise match
    if testcase["Enterprise"].lower() == bank_profile["size"].lower():
        score += w_enterprise * 100
    elif bank_profile["size"].lower() == "big" and testcase["Enterprise"] == "Small":
        # big bank can handle small enterprises more easily
        score += w_enterprise * 80
    else:
        score += w_enterprise * 30
    
    # Budget match (bank budget pref vs testcase budget)
    pref = bank_profile["budget_pref"].lower()
    tb = testcase["Budget"].lower()
    if pref == "high" and tb == "high":
        score += w_budget * 100
    elif pref == "medium" and tb in ("high","medium"):
        score += w_budget * 90
    elif pref == "low" and tb == "low":
        score += w_budget * 100
    else:
        # partial credit
        score += w_budget * 40
    
    # Infra match
    if testcase["Infra"].lower() == bank_profile["hw_pref"].lower():
        score += w_infra * 100
    else:
        # if bank has 'latest' treat as new
        if bank_profile["hw_pref"].lower() in ("latest","new") and testcase["Infra"].lower()=="old":
            score += w_infra * 30
        else:
            score += w_infra * 50
    
    # Legacy presence: if testcase has legacy and bank hw is old => better fit
    if testcase["Legacy"] == "Present":
        if bank_profile["hw_pref"] in ("old",):
            score += w_legacy * 100
        else:
            score += w_legacy * 40
    else:
        score += w_legacy * 80
    
    # Deployment: same server vs different servers (affects centralization)
    if testcase["Deployment"] == "Same server":
        # big banks prefer same server
        if bank_profile["size"] == "Big":
            score += w_deployment * 100
        else:
            score += w_deployment * 70
    else:
        score += w_deployment * 60
    
    # Direction & bank relation: small weight here
    score += w_direction * 80
    score += w_bank_relation * 80
    
    # Policy alignment:
    # If bank is adopt_pqc and testcase infra/budget supports PQC -> good
    policy = bank_profile["policy"]
    if policy == "adopt_pqc":
        if testcase["Infra"] == "New" and testcase["Budget"] in ("High","Medium"):
            score += w_policy * 100
        else:
            score += w_policy * 60
    elif policy == "hybrid":
        # hybrid banks are flexible
        score += w_policy * 90
    elif policy == "classical_only":
        # classical-only banks prefer legacy setups
        if testcase["Legacy"] == "Present" or testcase["Infra"]=="Old" or testcase["Budget"]=="Low":
            score += w_policy * 95
        else:
            score += w_policy * 30
    elif policy == "classical_but_latest_hw":
        # hardware is latest but policy classical -> medium
        if testcase["Infra"]=="New" or testcase["Budget"] in ("High","Medium"):
            score += w_policy * 85
        else:
            score += w_policy * 50
    else:
        score += w_policy * 50
    
    return min(100, score)

# -------------------------
# 5) Recommendation engine for an individual (test-case, bank)
# -------------------------
def recommend_for_case_bank(testcase, bank_name, bank_profile):
    rec = {}
    score = compatibility_score(testcase, bank_profile)
    rec["compatibility_score"] = round(score,2)
    reasons = []
    
    # High-level recommendation (software)
    policy = bank_profile["policy"]
    infra = testcase["Infra"]
    budget_v = testcase["Budget"]
    legacy = testcase["Legacy"]
    deployment = testcase["Deployment"]
    
    sw_recs = []
    hw_recs = []
    
    # Decide software recommendation
    if policy == "adopt_pqc":
        # full PQC adoption
        # pick PQC algos prioritized by budget/infra
        if budget_v == "High" and infra=="New":
            sw_recs = ["Falcon (Signature)", "Kyber (KEM)", "Dilithium (Signature)"]
            reasons.append("Bank is PQC-adopter and test-case has High budget + New infra -> recommend full PQC (Falcon/Dilithium/Kyber).")
        elif budget_v == "Medium":
            sw_recs = ["Kyber (KEM)", "Dilithium (Signature)"]
            reasons.append("Medium budget -> recommend balanced PQC (Kyber, Dilithium).")
        else:
            sw_recs = ["Kyber (KEM) - optimize implementation"]
            reasons.append("Low budget -> recommend Kyber (KEM) optimized for resource constraints.")
    elif policy == "hybrid":
        # hybrid: prefer hybrid stack
        sw_recs = HYBRID_RECOMMENDATION.copy()
        reasons.append("Hybrid policy -> recommend hybrid classical+PQC stack.")
        if legacy=="Present":
            reasons.append("Legacy present -> keep classical for compatibility and add PQC fallback.")
    elif policy == "classical_only":
        # classical only
        sw_recs = ["RSA/ECDSA (classical)"]
        reasons.append("Bank policy classical-only -> default to classical algorithms.")
        # but if test-case infra/new & budget high suggest optional PQC trial
        if infra=="New" and budget_v in ("High","Medium"):
            sw_recs.append("Optional: Kyber (pilot)")
            reasons.append("New infra + budget allows optional PQC pilot.")
    elif policy == "classical_but_latest_hw":
        # policy classical but hardware latest -> optional PQC
        sw_recs = ["RSA/ECDSA (classical)"]
        reasons.append("Policy classical but latest hardware -> classical preferred with PQC possible.")
        if budget_v in ("High","Medium") or infra=="New":
            sw_recs.append("Optional: Dilithium / Kyber (enable on hardware)")
            reasons.append("Hardware capable -> suggest optional PQC deployment.")
    else:
        sw_recs = ["RSA/ECDSA"]
        reasons.append("Default fallback to classical.")

    # Decide hardware recommendation
    # Map bank hw_pref and testcase constraints into a hw template
    bank_hw_pref = bank_profile["hw_pref"]
    if bank_hw_pref in ("new","latest"):
        # support high-end
        if budget_v == "High":
            hw_recs.append("high_end_fpga")
            reasons.append("High budget + new/latest hardware -> high-end FPGA/ASIC recommended.")
        elif budget_v == "Medium":
            hw_recs.append("mid_tier_fpga")
            reasons.append("Medium budget -> mid-tier FPGA/accelerator recommended.")
        else:
            hw_recs.append("cpu_with_accel")
            reasons.append("Low budget -> server CPU with crypto acceleration preferred.")
    elif bank_hw_pref == "mid":
        if budget_v == "High":
            hw_recs.append("mid_tier_fpga")
            reasons.append("Mid hardware + high budget -> mid-tier FPGA.")
        else:
            hw_recs.append("cpu_with_accel")
            reasons.append("Mid hardware and budget -> CPU with acceleration.")
    elif bank_hw_pref == "old":
        hw_recs.append("lightweight_cpu")
        reasons.append("Old hardware -> lightweight CPU or minimal upgrade.")
    else:
        hw_recs.append("cpu_with_accel")
        reasons.append("Default to CPU with acceleration.")

    # If deployment is Different servers, prefer cpu_with_accel to avoid per-server hardware changes
    if testcase["Deployment"] == "Different servers" and "cpu_with_accel" not in hw_recs:
        hw_recs.insert(0, "cpu_with_accel")
        reasons.append("Different servers deployment -> CPU-accelerated approach preferred for portability.")

    # Build final descriptive strings
    # Choose first hardware template as primary
    hw_primary = hw_recs[0]
    hw_desc = HW_MODULE_TEMPLATES.get(hw_primary, {"desc":hw_primary,"suitable_for":""})["desc"]
    hw_suitable = HW_MODULE_TEMPLATES.get(hw_primary, {}).get("suitable_for", "")
    sw_primary = sw_recs[0] if sw_recs else "None"

    rec["recommended_algorithms"] = "; ".join(sw_recs)
    rec["recommended_hw_module_key"] = hw_primary
    rec["recommended_hw_module_desc"] = hw_desc
    rec["hw_suitable_for"] = hw_suitable
    rec["reasons"] = " | ".join(reasons)

    # Add compatibility flags for quick filtering
    rec["bank_policy"] = bank_profile["policy"]
    rec["bank_hw_pref"] = bank_profile["hw_pref"]
    rec["bank_size"] = bank_profile["size"]
    rec["bank_name"] = bank_name

    return rec

# -------------------------
# 6) Apply mapping to all test cases × banks
# -------------------------
rows = []
for i, tc in test_cases_df.iterrows():
    tc_dict = tc.to_dict()
    for bank_name, bank_profile in bank_profiles.items():
        rec = recommend_for_case_bank(tc_dict, bank_name, bank_profile)
        # Flatten row: include test case columns followed by rec fields
        row = {**tc_dict, **rec}
        rows.append(row)

mapping_df = pd.DataFrame(rows)
print("Mapping rows:", mapping_df.shape)  # expect (192*4, ?)

# -------------------------
# 7) Post-process: derive simple flags & scoring buckets
# -------------------------
def score_bucket(s):
    if s >= 80:
        return "High"
    elif s >= 60:
        return "Medium"
    else:
        return "Low"

mapping_df["compatibility_bucket"] = mapping_df["compatibility_score"].apply(score_bucket)

# Reorder columns for readability
front_cols = columns + ["bank_name", "bank_policy", "bank_size", "bank_hw_pref", "compatibility_score", "compatibility_bucket"]
rear_cols = ["recommended_algorithms", "recommended_hw_module_key", "recommended_hw_module_desc", "hw_suitable_for", "reasons"]
final_cols = front_cols + rear_cols
mapping_df = mapping_df[final_cols]

# -------------------------
# 8) Save & preview
# -------------------------
mapping_df.to_csv("pqc_testcases_bank_mapping.csv", index=False)
print("Saved pqc_testcases_bank_mapping.csv with shape:", mapping_df.shape)

print("\nSample rows:")
print(mapping_df.head(12))

# Also show summary counts per bank and bucket
print("\nCompatibility bucket counts by bank:")
print(mapping_df.groupby(["bank_name", "compatibility_bucket"]).size().unstack(fill_value=0))

# Optionally: show recommended HW distribution
print("\nTop recommended hardware per bank:")
print(mapping_df.groupby("bank_name")["recommended_hw_module_key"].value_counts().groupby(level=0).head(10))


Generated test case count: 192
Mapping rows: (768, 17)
Saved pqc_testcases_bank_mapping.csv with shape: (768, 18)

Sample rows:
   Enterprise Budget Infra   Legacy   Deployment     Direction  \
0         Big   High   New  Present  Same server  User->Server   
1         Big   High   New  Present  Same server  User->Server   
2         Big   High   New  Present  Same server  User->Server   
3         Big   High   New  Present  Same server  User->Server   
4         Big   High   New  Present  Same server  User->Server   
5         Big   High   New  Present  Same server  User->Server   
6         Big   High   New  Present  Same server  User->Server   
7         Big   High   New  Present  Same server  User->Server   
8         Big   High   New  Present  Same server  Server->User   
9         Big   High   New  Present  Same server  Server->User   
10        Big   High   New  Present  Same server  Server->User   
11        Big   High   New  Present  Same server  Server->User   

     Bank rel

In [3]:
import itertools
import pandas as pd

# ------------------------
# Bank definitions
# ------------------------
banks = {
    "ICICI": {"strength": 3, "algo": "PQC (Kyber/Dilithium/Falcon)"},
    "HDFC": {"strength": 2, "algo": "Hybrid (RSA+Kyber/Dilithium)"},
    "SBI": {"strength": 1, "algo": "Classical (RSA/ECDSA)"},
    "Saraswat": {"strength": 2, "algo": "Classical (RSA/ECDSA) + Optional PQC"},
}

# Parameter sets
enterprise = ["Big", "Small"]
budget = ["High", "Medium", "Low"]
infra = ["New", "Old"]
legacy = ["Present", "Absent"]

# ------------------------
# Helper: upgrade/downgrade logic
# ------------------------
def resolve_algos(sender, receiver):
    s_strength = banks[sender]["strength"]
    r_strength = banks[receiver]["strength"]
    s_algo = banks[sender]["algo"]
    r_algo = banks[receiver]["algo"]

    if s_strength > r_strength:
        # downgrade at receiver
        resolved = f"{r_algo} (downgraded)"
    elif s_strength < r_strength:
        # upgrade at receiver
        resolved = f"{r_algo} (upgraded)"
    else:
        resolved = r_algo
    return s_algo, resolved

# ------------------------
# Generate test cases
# ------------------------
rows = []
for (sender, receiver) in itertools.permutations(banks.keys(), 2):  # ordered pairs, no self
    deployment = "Same server" if sender == receiver else "Different servers"
    for ent, bud, inf, leg in itertools.product(enterprise, budget, infra, legacy):
        s_algo, r_algo = resolve_algos(sender, receiver)
        rows.append({
            "From Bank": sender,
            "To Bank": receiver,
            "Enterprise": ent,
            "Budget": bud,
            "Infra": inf,
            "Legacy": leg,
            "Deployment": deployment,
            "Sender Algo": s_algo,
            "Receiver Algo": r_algo
        })

df = pd.DataFrame(rows)
print("Total test cases:", len(df))  # should be 288
print(df.head(10))

# Save CSV
df.to_csv("bank_to_bank_testcases.csv", index=False)
print("Saved to bank_to_bank_testcases.csv")


Total test cases: 288
  From Bank To Bank Enterprise  Budget Infra   Legacy         Deployment  \
0     ICICI    HDFC        Big    High   New  Present  Different servers   
1     ICICI    HDFC        Big    High   New   Absent  Different servers   
2     ICICI    HDFC        Big    High   Old  Present  Different servers   
3     ICICI    HDFC        Big    High   Old   Absent  Different servers   
4     ICICI    HDFC        Big  Medium   New  Present  Different servers   
5     ICICI    HDFC        Big  Medium   New   Absent  Different servers   
6     ICICI    HDFC        Big  Medium   Old  Present  Different servers   
7     ICICI    HDFC        Big  Medium   Old   Absent  Different servers   
8     ICICI    HDFC        Big     Low   New  Present  Different servers   
9     ICICI    HDFC        Big     Low   New   Absent  Different servers   

                    Sender Algo                              Receiver Algo  
0  PQC (Kyber/Dilithium/Falcon)  Hybrid (RSA+Kyber/Dilithium) (d

In [4]:
import itertools
import pandas as pd

# ------------------------
# Bank profiles
# ------------------------
banks = {
    "ICICI": {
        "strength": 3,
        "algo": "PQC (Kyber/Dilithium/Falcon)",
        "hardware": "New",
        "infra": "Big",
        "enterprise": "Big"
    },
    "HDFC": {
        "strength": 2,
        "algo": "Hybrid (RSA+Kyber/Dilithium)",
        "hardware": "Mid",
        "infra": "Big",
        "enterprise": "Big"
    },
    "SBI": {
        "strength": 1,
        "algo": "Classical (RSA/ECDSA)",
        "hardware": "Old",
        "infra": "Low",
        "enterprise": "Big"
    },
    "Saraswat": {
        "strength": 2,
        "algo": "Classical (RSA/ECDSA) + Latest HW",
        "hardware": "Latest",
        "infra": "Mid",
        "enterprise": "Small"
    }
}

budgets = ["High", "Medium", "Low"]
legacy_options = ["Present", "Absent"]

# ------------------------
# Algo resolution (upgrade/downgrade)
# ------------------------
def resolve_algos(sender, receiver):
    s_strength = banks[sender]["strength"]
    r_strength = banks[receiver]["strength"]
    s_algo = banks[sender]["algo"]
    r_algo = banks[receiver]["algo"]

    if s_strength > r_strength:
        resolved = f"{r_algo} (downgraded)"
    elif s_strength < r_strength:
        resolved = f"{r_algo} (upgraded)"
    else:
        resolved = r_algo
    return s_algo, resolved

# ------------------------
# Generate test cases
# ------------------------
rows = []
for sender, receiver in itertools.product(banks.keys(), banks.keys()):  # includes same-bank
    deployment = "Same server" if sender == receiver else "Different servers"
    for budget, legacy in itertools.product(budgets, legacy_options):
        s_algo, r_algo = resolve_algos(sender, receiver)
        rows.append({
            "From Bank": sender,
            "To Bank": receiver,
            "Enterprise": banks[sender]["enterprise"],
            "Infra": banks[sender]["infra"],
            "Hardware": banks[sender]["hardware"],
            "Budget": budget,
            "Legacy": legacy,
            "Deployment": deployment,
            "Sender Algo": s_algo,
            "Receiver Algo": r_algo
        })

df = pd.DataFrame(rows)
print("Total test cases:", len(df))  # 96
print(df.head(10))

# Save
df.to_csv("bank_testcases.csv", index=False)
print("Saved to bank_testcases.csv")


Total test cases: 96
  From Bank To Bank Enterprise Infra Hardware  Budget   Legacy  \
0     ICICI   ICICI        Big   Big      New    High  Present   
1     ICICI   ICICI        Big   Big      New    High   Absent   
2     ICICI   ICICI        Big   Big      New  Medium  Present   
3     ICICI   ICICI        Big   Big      New  Medium   Absent   
4     ICICI   ICICI        Big   Big      New     Low  Present   
5     ICICI   ICICI        Big   Big      New     Low   Absent   
6     ICICI    HDFC        Big   Big      New    High  Present   
7     ICICI    HDFC        Big   Big      New    High   Absent   
8     ICICI    HDFC        Big   Big      New  Medium  Present   
9     ICICI    HDFC        Big   Big      New  Medium   Absent   

          Deployment                   Sender Algo  \
0        Same server  PQC (Kyber/Dilithium/Falcon)   
1        Same server  PQC (Kyber/Dilithium/Falcon)   
2        Same server  PQC (Kyber/Dilithium/Falcon)   
3        Same server  PQC (Kyber/Dil

In [5]:
import pandas as pd
import itertools

# -----------------------------
# 1. Load datasets
# -----------------------------
sw_df = pd.read_csv("kyber_dilithium_rsa_dataset - Copy.csv", encoding="latin1")
hw_df = pd.read_csv("hwd_fpga_final.csv")

# Normalize numeric cols for resource scoring
numeric_cols = ['Max Freq (MHz)', 'LUT', 'FF', 'DSP', 'BRAM']
for col in numeric_cols:
    hw_df[col] = pd.to_numeric(hw_df[col], errors='coerce')
hw_df['resource_score'] = hw_df[numeric_cols].apply(lambda x: (x - x.min())/(x.max() - x.min()), axis=0).mean(axis=1)

# -----------------------------
# 2. Bank Profiles
# -----------------------------
banks = {
    "ICICI": {"policy": "pqc", "hardware": "new", "size": "Big", "strength": 3},
    "HDFC": {"policy": "hybrid", "hardware": "mid", "size": "Big", "strength": 2},
    "SBI": {"policy": "classical", "hardware": "old", "size": "Big", "strength": 1},
    "Saraswat": {"policy": "classical", "hardware": "latest", "size": "Small", "strength": 2},
}

enterprise_opts = ["Big", "Small"]
budget_opts = ["High", "Medium", "Low"]
infra_opts = ["New", "Old"]
legacy_opts = ["Present", "Absent"]

# -----------------------------
# 3. Algorithm Selection Function
# -----------------------------
def select_algo(bank, budget, infra, legacy, hw_score):
    policy = banks[bank]["policy"]

    if policy == "pqc":
        if budget == "High" and infra == "New" and hw_score > 0.6:
            return "Falcon"
        elif budget in ["High", "Medium"] and hw_score > 0.4:
            return "Dilithium"
        else:
            return "Kyber"

    elif policy == "hybrid":
        if legacy == "Present":
            return "Kyber"
        elif budget == "High" and hw_score > 0.4:
            return "Dilithium"
        else:
            return "Kyber"

    elif policy == "classical":
        if infra == "New" and budget == "High" and hw_score > 0.4:
            return "Dilithium"
        elif legacy == "Present":
            return "SPHINCS+"
        else:
            return "RSA"

    return "RSA"

# -----------------------------
# 4. Hardware Selector
# -----------------------------
def select_hardware(algo):
    if algo == "Falcon":
        candidates = hw_df[hw_df['resource_score'] > 0.6]
    elif algo == "Dilithium":
        candidates = hw_df[hw_df['resource_score'] > 0.4]
    elif algo == "SPHINCS+":
        candidates = hw_df[hw_df['resource_score'] > 0.2]
    else:  # Kyber or RSA fallback
        candidates = hw_df

    if candidates.empty:
        return {"Design": "Generic CPU", "resource_score": 0}
    best = candidates.sort_values(by=["resource_score", "Max Freq (MHz)"], ascending=False).iloc[0]
    return {"Design": best["Design"], "resource_score": best["resource_score"]}

# -----------------------------
# 5. Generate Test Cases
# -----------------------------
cases = []
for from_bank, to_bank in itertools.product(banks.keys(), repeat=2):
    deployment = "Same server" if from_bank == to_bank else "Different servers"

    for enterprise, budget, infra, legacy in itertools.product(enterprise_opts, budget_opts, infra_opts, legacy_opts):

        # pick sender hardware score as mean for simplicity
        avg_hw_score = hw_df['resource_score'].mean()

        # sender algo
        sender_algo = select_algo(from_bank, budget, infra, legacy, avg_hw_score)

        # receiver algo: upgrade/downgrade rule
        if banks[from_bank]["strength"] > banks[to_bank]["strength"]:
            receiver_algo = select_algo(to_bank, budget, infra, legacy, avg_hw_score)
        elif banks[from_bank]["strength"] < banks[to_bank]["strength"]:
            receiver_algo = select_algo(to_bank, budget, infra, legacy, avg_hw_score)
        else:
            receiver_algo = select_algo(to_bank, budget, infra, legacy, avg_hw_score)

        # hardware
        sender_hw = select_hardware(sender_algo)
        receiver_hw = select_hardware(receiver_algo)

        cases.append({
            "From Bank": from_bank,
            "To Bank": to_bank,
            "Enterprise": enterprise,
            "Budget": budget,
            "Infra": infra,
            "Legacy": legacy,
            "Deployment": deployment,
            "Sender Algo": sender_algo,
            "Receiver Algo": receiver_algo,
            "Sender_HW_Design": sender_hw["Design"],
            "Sender_HW_resource_score": sender_hw["resource_score"],
            "Receiver_HW_Design": receiver_hw["Design"],
            "Receiver_HW_resource_score": receiver_hw["resource_score"]
        })

# -----------------------------
# 6. Save Results
# -----------------------------
test_cases_df = pd.DataFrame(cases)
print("Total test cases generated:", len(test_cases_df))
print(test_cases_df.head(10))

test_cases_df.to_csv("pqc_bank_testcases.csv", index=False)
print("Saved full test case matrix to pqc_bank_testcases.csv")


Total test cases generated: 384
  From Bank To Bank Enterprise  Budget Infra   Legacy   Deployment  \
0     ICICI   ICICI        Big    High   New  Present  Same server   
1     ICICI   ICICI        Big    High   New   Absent  Same server   
2     ICICI   ICICI        Big    High   Old  Present  Same server   
3     ICICI   ICICI        Big    High   Old   Absent  Same server   
4     ICICI   ICICI        Big  Medium   New  Present  Same server   
5     ICICI   ICICI        Big  Medium   New   Absent  Same server   
6     ICICI   ICICI        Big  Medium   Old  Present  Same server   
7     ICICI   ICICI        Big  Medium   Old   Absent  Same server   
8     ICICI   ICICI        Big     Low   New  Present  Same server   
9     ICICI   ICICI        Big     Low   New   Absent  Same server   

  Sender Algo Receiver Algo Sender_HW_Design  Sender_HW_resource_score  \
0       Kyber         Kyber         Kintex-7                  0.426336   
1       Kyber         Kyber         Kintex-7     

In [6]:
import itertools
import pandas as pd

# -------------------------
# Bank profiles (with strengths & crypto policies)
# -------------------------
banks = {
    "ICICI": {"strength": 3, "algo": "PQC", "infra": "new", "enterprise": "big"},
    "HDFC": {"strength": 2, "algo": "Hybrid", "infra": "mid", "enterprise": "big"},
    "SBI": {"strength": 1, "algo": "Classical", "infra": "old", "enterprise": "big"},
    "Saraswat": {"strength": 2, "algo": "Classical", "infra": "latest", "enterprise": "small"}
}

budgets = ["High", "Medium", "Low"]
legacy_options = ["Present", "Absent"]

# -------------------------
# Algo resolution (upgrade/downgrade)
# -------------------------
def resolve_algos(sender, receiver):
    s_strength = banks[sender]["strength"]
    r_strength = banks[receiver]["strength"]
    s_algo = banks[sender]["algo"]
    r_algo = banks[receiver]["algo"]

    if s_strength > r_strength:
        r_algo_resolved = r_algo + " (downgraded)"
    elif s_strength < r_strength:
        r_algo_resolved = r_algo + " (upgraded)"
    else:
        r_algo_resolved = r_algo
    return s_algo, r_algo_resolved

# -------------------------
# HW recommendation logic
# -------------------------
def recommend_hw(algo, budget, infra):
    # Base on software complexity
    if "PQC" in algo:
        base = "High-performance FPGA/ASIC"
    elif "Hybrid" in algo:
        base = "Mid-tier FPGA + CPU combo"
    else:
        base = "Commodity CPU or Low-end FPGA"

    # Modify by budget
    if budget == "High":
        budget_hw = "Enterprise HSM + FPGA"
    elif budget == "Medium":
        budget_hw = "Shared FPGA + CPU cluster"
    else:
        budget_hw = "Low-cost CPU only"

    # Modify by infra
    if infra in ["new", "latest"]:
        infra_hw = "Supports advanced distributed crypto infra"
    elif infra == "mid":
        infra_hw = "Supports moderate hybrid infra"
    else:
        infra_hw = "Legacy infra with CPU-only support"

    return base, budget_hw, infra_hw

# -------------------------
# Generate test cases
# -------------------------
rows = []
for sender, receiver in itertools.product(banks.keys(), banks.keys()):
    deployment = "Same server" if sender == receiver else "Different servers"
    for budget, legacy in itertools.product(budgets, legacy_options):
        s_algo, r_algo = resolve_algos(sender, receiver)

        base_hw, budget_hw, infra_hw = recommend_hw(s_algo, budget, banks[sender]["infra"])

        rows.append({
            "From Bank": sender,
            "To Bank": receiver,
            "Enterprise": banks[sender]["enterprise"],
            "Infra": banks[sender]["infra"],
            "Budget": budget,
            "Legacy": legacy,
            "Deployment": deployment,
            "Sender Algo": s_algo,
            "Receiver Algo": r_algo,
            "HW_Base": base_hw,
            "HW_Budget": budget_hw,
            "HW_Infra": infra_hw
        })

df = pd.DataFrame(rows)
print("Total test cases:", len(df))  # 96
print(df.head(10))

# Save
df.to_csv("bank_testcases.csv", index=False)
print("Saved to bank_testcases.csv")


Total test cases: 96
  From Bank To Bank Enterprise Infra  Budget   Legacy         Deployment  \
0     ICICI   ICICI        big   new    High  Present        Same server   
1     ICICI   ICICI        big   new    High   Absent        Same server   
2     ICICI   ICICI        big   new  Medium  Present        Same server   
3     ICICI   ICICI        big   new  Medium   Absent        Same server   
4     ICICI   ICICI        big   new     Low  Present        Same server   
5     ICICI   ICICI        big   new     Low   Absent        Same server   
6     ICICI    HDFC        big   new    High  Present  Different servers   
7     ICICI    HDFC        big   new    High   Absent  Different servers   
8     ICICI    HDFC        big   new  Medium  Present  Different servers   
9     ICICI    HDFC        big   new  Medium   Absent  Different servers   

  Sender Algo        Receiver Algo                     HW_Base  \
0         PQC                  PQC  High-performance FPGA/ASIC   
1         